In [1]:
import json
import json_coder
import pandas as pd
import networkx as nx
import befaas as bf
import befaas.logentry as le
from befaas.graph import build_function_graph, add_default_metadata, conv_to_ms

filepath = "../dumps/"
logdumps = ["dump_iot_google.json","dump_iot_aws.json","dump_iot_azure.json"]
#logdumps = ["dump_iot_google.json"]
outfile = "../img/latenciesIoT.pkl"
plotdata = []

print("Everything is set up.")

Everything is set up.


In [2]:
for dump in logdumps:
    print(f"Include dump {dump} ...")
    print("Load dump... (may take a while)")
    data = bf.load_logs(filepath + dump)
    print("Done.")

    print("Sort data to contexts...")
    contexts = {}

    for entry in data:
        id = entry.context_id
        if id == None:
            continue

        if not (id in contexts):
            contexts[id] = []

        contexts[id].append(entry)
    print("Done.")

    print("Find xpairs...")
    xpairs = {}

    for ctx_id in contexts:
        ctx = contexts[ctx_id]
        for entry in ctx:
            id = entry.event["xPair"]
            if not (id in xpairs):
                xpairs[id] = []

            specialPair = False
            if isinstance(entry, le.PerfLog):
                if not entry.perf_type_data == "":
                    parts = entry.perf_type_data.split(":")
                    if len(parts) > 1:
                        id = parts[1]
                        if not (id in xpairs):
                            xpairs[id] = []
                        xpairs[id].append(entry)
                        specialPair = True

            if not specialPair:
                xpairs[id].append(entry)
    print("done.")

    print("Calculate transport times...")
    debugpairs = []
    debugpairs2 = []

    for xpairID in xpairs:
        xpair = xpairs[xpairID]

        src = ""
        dst = ""
        platform = ""
        latency = 0
        start = None
        type = ""
        hasDB = False

        for entry in xpair:
            if isinstance(entry, le.ArtilleryLog):
                type = "Load"
                break

        if type == "":
            for entry in xpair:
                if isinstance(entry, le.PerfLog):
                    if entry.perf_type[1] == "dbOut":
                        hasDB = True
                        break

        if type == "":
            rpcInFound = False
            rpcOutFound = False

            for entry in xpair:
                if isinstance(entry, le.PerfLog):
                    if entry.type == "measure":
                        if entry.perf_type[1] == "rpcIn":
                            rpcInFound = True
                        if entry.perf_type[1] == "rpcOut":
                            rpcOutFound = True

            if rpcInFound and rpcOutFound:
                type = "Call"

        if type == "Load":
            src = "Artillery"
            platform = "AWS → tinyFaaS"
            artilleryStart = None
            artilleryEnd = None
            duration = 0

            for entry in xpair:
                if isinstance(entry, le.ArtilleryLog):
                    if (entry.event["type"] == "before"):
                        artilleryStart = entry.timestamp

                    if (entry.event["type"] == "after"):
                        artilleryEnd = entry.timestamp

                if isinstance(entry, le.PerfLog):
                    if entry.type == "measure":
                        if entry.perf_type[1] == "post":
                            duration = entry.perf["duration"]
                            dst = entry.function
                        if entry.perf_type[1] == "rpcIn":
                            duration = entry.perf["duration"]
                            dst = entry.function

            latency = (artilleryEnd.timestamp() - artilleryStart.timestamp()) * 1000 # to ms
            latency -= duration
            start = artilleryStart

            # print(f"Load: {src} - {dst} | {platform} | {latency}")

        if type == "Call":
            srcDuration = 0
            dstDuration = 0
            for entry in xpair:
                if isinstance(entry, le.PerfLog):
                    if not entry.platform == "tinyfaas":
                        platform = entry.platform
                    if entry.type == "measure":
                        if entry.perf_type[1] == "rpcIn":
                            dstDuration = entry.perf["duration"]
                            dst = entry.function

                        if entry.perf_type[1] == "rpcOut":
                            srcDuration += entry.perf["duration"]
                            src = entry.function
                    if entry.type == "mark":
                        if entry.perf_type[1] == "rpcOut":
                            start = entry.timestamp

            latency = srcDuration - dstDuration

            if latency > 10000:
                debugpairs2.append(xpair)
            # print(f"Call: {src} - {dst} | {latency}")

        if type == "":
            debugpairs.append(xpair)

        if src == "":
            print(f"No SRC: {src} - {dst} - {type} - {latency}")
        else:
            if not dst == "DB" and not src.__contains__("roadcondition") and (dst.__contains__("lightphase") or dst.__contains__("statistics")):
                # print(f"Call: {src} - {dst} | {latency} | {platform} | {start}")
                row = {}
                row["pair"] = src + " - " + dst
                row["latency"] = latency
                row["platform"] = "tinyfaas - " +  platform
                row["start"] = start
                plotdata.append(row)



Include dump dump_iot_google.json ...
Load dump... (may take a while)
Done.
Sort data to contexts...
Done.
Find xpairs...
done.
Calculate transport times...
Include dump dump_iot_aws.json ...
Load dump... (may take a while)
Done.
Sort data to contexts...
Done.
Find xpairs...
done.
Calculate transport times...
Include dump dump_iot_azure.json ...
Load dump... (may take a while)
Done.
Sort data to contexts...
Done.
Find xpairs...
done.
Calculate transport times...


In [3]:
df_calls = pd.DataFrame(plotdata)
df_calls.head()
df_calls.describe()

,latency
count,4496.000000
mean,104.202265
std,58.052359
min,58.793095
25%,78.984089
50%,88.269431
75%,133.648789
max,1827.088460


In [4]:
df_calls.to_pickle(outfile)